# Angle of incidence (AOI)

### Importing the necessary packages

In [ ]:
import numpy as np
import pandas as pd
import pvlib

### Defining the paramteres

In [ ]:
lat = 59.97315    # Latitude of the test site
lon = 11.05166    # Longitude of the test site
tz = 0            # Wanted timezone, UTC 
alt = 214         # Altitude
loc = 'Kjeller'   # Location     

tilt = 90         # Tilt of solar panels
az_e = 86         # Azimuth towards east
az_w = 266        # Azimuth towards west

### Calculating the AOI and corresponding (incident angle modifier) IAM

In [ ]:
data = pd.read_csv(r'path...\filename.csv')    # Importing the file with simulated irradiance from bifacial_radiance
data = data.set_index('Unnamed: 0')            # Setting the index
data.index = pd.to_datetime(data.index)        # Setting the index to be date-time

site = pvlib.location.Location(lat, lon, tz, alt, loc)    # Defining the location
ephem_data = site.get_solarposition(data.index)           # Getting data about the solar position 

solar_zenith = ephem_data['zenith']      # Selecting the zenith angles
solar_azimuth = ephem_data['azimuth']    # selecting the azimuth angles

# Calculating the AOI for reference values
aoi_east = pvlib.irradiance.aoi(tilt, az_e, solar_zenith, solar_azimuth)   
aoi_west = pvlib.irradiance.aoi(tilt, az_w, solar_zenith, solar_azimuth)

# Calculating the incidence angle modifier (IAM) for reference values
iam = pvlib.iam.ashrae(aoi_east)                                           
iam = pvlib.iam.ashrae(aoi_west)

# Calculating IAM for all aoi angles
iam_east = pvlib.iam.interp(aoi_east,
                            [75.7835, 62.1417, 48.8441, 36.384, 63.2874, 76.7623],         # Reference angles for AOI
                            [0.846406, 0.942999, 0.974025, 0.987693, 0.938769, 0.831635])  # Reference values for IAM 

iam_west = pvlib.iam.interp(aoi_west,
                            [75.7835, 62.1417, 48.8441, 36.384, 63.2874, 76.7623], 
                            [0.846406, 0.942999, 0.974025, 0.987693, 0.938769, 0.831635])

### Taking losses due to AOI into account for the simulated irradiance by multplying with IAM

In [ ]:
data['east_aoi'] = data['column_name']*iam_east
data['west_aoi'] = data['column_name']*iam_west

The same procedure was used to calculate the AOI for the upper and lower limit of irradiance when the uncertainty in the measurements was taken into account.